In [223]:
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import torch
from torchsummary import summary
import torchvision
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import torchvision.transforms as transforms
from torch import nn, sigmoid
from torch.nn.functional import relu

In [148]:
df = pd.read_pickle('F:\projects\dna_ai\data\data.pkl')
df = df.sample(frac = 1, ignore_index=True) # перемешаем
df.head()

,Masked sequence,Y1,Y2,Count target,Start,End
0,"[[False, False, False, True, False], [False, F...",False,"[False, False, False, False, False, False, Fal...",0,None,None
1,"[[False, True, False, False, False], [False, T...",True,"[False, False, False, False, False, False, Fal...",1,[32853078],[32853166]
2,"[[False, False, False, True, False], [True, Fa...",True,"[False, False, False, False, False, False, Fal...",1,[10440346],[10440413]
3,"[[False, False, False, True, False], [False, F...",True,"[False, False, False, False, False, False, Fal...",1,[217862773],[217862854]
4,"[[False, False, False, True, False], [False, F...",True,"[False, False, False, False, False, False, Fal...",1,[108648565],[108648991]


In [153]:
df['Masked sequence'][1]

array([[False,  True, False, False, False],
       [False,  True, False, False, False],
       [False,  True, False, False, False],
       ...,
       [False, False, False,  True, False],
       [False, False, False,  True, False],
       [False,  True, False, False, False]])

In [152]:
df['Masked sequence'][1].shape

(1000, 5)

In [154]:
n, m = df['Masked sequence'][1].shape

In [161]:
df['Masked sequence'][1].T

array([[False, False, False, ..., False, False, False],
       [ True,  True,  True, ..., False, False,  True],
       [False, False, False, ..., False, False, False],
       [False, False, False, ...,  True,  True, False],
       [False, False, False, ..., False, False, False]])

In [158]:
np.reshape(df['Masked sequence'][1], (-1, 1))

array([[False],
       [ True],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [156]:
np.reshape(df['Masked sequence'][1], (m, n)).shape

(5, 1000)

In [3]:
df.shape

(39372, 6)

In [18]:
# from sklearn.model_selection import train_test_split

# # Разделение на тренировочную и тестовую выборки
# x_train, x_test, y_train, y_test = train_test_split(df['Masked sequence'], df['Y2'], test_size=0.2)
# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25)

# # Вывод размерностей тренировочной и тестовой выбороки
# print("Размерность тренировочной выборки:", x_train.shape)
# print("Размерность тестовой выборки:", x_test.shape)
# print("Размерность маски теста:", y_test.iloc[0].shape)
# print("Размерность валидации при обучении:", x_val.shape)

Размерность тренировочной выборки: (23622,)
Размерность тестовой выборки: (7875,)
Размерность маски теста: (1000,)
Размерность валидации при обучении: (7875,)


In [237]:
if torch.cuda.is_available():     # Make sure GPU is available
    device = torch.device("cuda:0")
else:
    print("Warning: CUDA not found, CPU only.")
    device = torch.device("cpu")

In [240]:
class SequenceDataset(Dataset):
    def __init__(self, path):
        self.df = pd.read_pickle(path)
        self.df = df.sample(frac = 1, ignore_index=True)

    def __getitem__(self, index):
        x = self.df['Masked sequence'][index].T
        y = self.df['Y2'][index]
        return x, y
    
    def __len__(self):
        return len(self.df)


In [163]:
data = SequenceDataset('F:\projects\dna_ai\data\data.pkl')
batch_size = 128

In [38]:
len(data)

39372

In [243]:
from torch.utils.data import random_split
train_size = int(0.8 * len(data))
valid_size = len(data) - train_size
print(train_size, valid_size)

train_dataset, valid_dataset = random_split(data, [train_size, valid_size])

31497 7875


In [244]:
train_dataset, valid_dataset

(<torch.utils.data.dataset.Subset at 0x1ed04e824d0>,
 <torch.utils.data.dataset.Subset at 0x1ed04e82bf0>)

In [245]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)

In [246]:
len(train_loader), len(valid_loader)

(247, 62)

In [250]:
len(next(iter(train_loader)))

2

In [230]:
class BaumanNet(nn.Module):
    def __init__(self):
        super(BaumanNet, self).__init__()
        
        self.conv11 = nn.Conv1d(in_channels=5, out_channels=16, kernel_size=1, padding= 'same')
        "Encoder"
        self.conv12 = nn.Conv1d(in_channels=16, out_channels=16, kernel_size=3, padding=1)
        self.maxpool1 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv21 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.conv22 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv31 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv32 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.maxpool3 = nn.MaxPool1d(kernel_size=2, stride=2)

        "Bottle neck"

        self.conv41 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv42 = nn.Conv1d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
        
        "Decoder"
        self.upconv5 = nn.ConvTranspose1d(in_channels=128, out_channels=64, kernel_size=2, stride=2)
        self.conv51 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=3, padding=1)
        self.conv52 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1)

        self.upconv6 = nn.ConvTranspose1d(in_channels=64, out_channels=32, kernel_size=2, stride=2)
        self.conv61 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, padding=1)
        self.conv62 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, padding=1)

        self.upconv7 = nn.ConvTranspose1d(in_channels=32, out_channels=16, kernel_size=2, stride=2)
        self.conv71 = nn.Conv1d(in_channels=32, out_channels=16, kernel_size=3, padding=1)
        self.conv72 = nn.Conv1d(in_channels=16, out_channels=16, kernel_size=3, padding=1)

        self.output = nn.Conv1d(in_channels=16, out_channels=1, kernel_size=1)
        
    def forward(self, x):
        
        xe11 = relu(self.conv11(x))
        "Encoder"
        xe12 = relu(self.conv12(xe11))
        xe13 = self.maxpool1(xe12)

        xe21 = relu(self.conv21(xe13))
        xe22 = relu(self.conv22(xe21))
        xe23 = self.maxpool2(xe22)

        xe31 = relu(self.conv31(xe23))
        xe32 = relu(self.conv32(xe31))
        xe33 = self.maxpool3(xe32)

        "Bottle neck"
        xe41 = relu(self.conv41(xe33))
        xe42 = relu(self.conv42(xe41))

        "Decoder"
        xe51 = self.upconv5(xe42)
        xe5 = torch.cat([xe51, xe32], dim=1)
        xe52 = relu(self.conv51(xe5))
        xe53 = relu(self.conv52(xe52))

        xe61 = self.upconv6(xe53)
        xe6 = torch.cat([xe61, xe22], dim=1)
        xe62 = relu(self.conv61(xe6))
        xe63 = relu(self.conv62(xe62))

        xe71 = self.upconv7(xe63)
        xe7 = torch.cat([xe71, xe12], dim=1)
        xe72 = relu(self.conv71(xe7))
        xe73 = relu(self.conv72(xe72))

    
        out = sigmoid(self.output(xe73))
        return out

In [239]:
model = BaumanNet().to(device)
# x, y = next(iter(train_loader))
summary(model, (5, 1000))

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv1d: 1-1                            [-1, 16, 1000]            96
├─Conv1d: 1-2                            [-1, 16, 1000]            784
├─MaxPool1d: 1-3                         [-1, 16, 500]             --
├─Conv1d: 1-4                            [-1, 32, 500]             1,568
├─Conv1d: 1-5                            [-1, 32, 500]             3,104
├─MaxPool1d: 1-6                         [-1, 32, 250]             --
├─Conv1d: 1-7                            [-1, 64, 250]             6,208
├─Conv1d: 1-8                            [-1, 64, 250]             12,352
├─MaxPool1d: 1-9                         [-1, 64, 125]             --
├─Conv1d: 1-10                           [-1, 128, 125]            24,704
├─Conv1d: 1-11                           [-1, 128, 125]            49,280
├─ConvTranspose1d: 1-12                  [-1, 64, 250]             16,448
├─Conv1d: 1-13                           [-1, 64, 250]     

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv1d: 1-1                            [-1, 16, 1000]            96
├─Conv1d: 1-2                            [-1, 16, 1000]            784
├─MaxPool1d: 1-3                         [-1, 16, 500]             --
├─Conv1d: 1-4                            [-1, 32, 500]             1,568
├─Conv1d: 1-5                            [-1, 32, 500]             3,104
├─MaxPool1d: 1-6                         [-1, 32, 250]             --
├─Conv1d: 1-7                            [-1, 64, 250]             6,208
├─Conv1d: 1-8                            [-1, 64, 250]             12,352
├─MaxPool1d: 1-9                         [-1, 64, 125]             --
├─Conv1d: 1-10                           [-1, 128, 125]            24,704
├─Conv1d: 1-11                           [-1, 128, 125]            49,280
├─ConvTranspose1d: 1-12                  [-1, 64, 250]             16,448
├─Conv1d: 1-13                           [-1, 64, 250]     

In [233]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.nn.functional.F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice

In [234]:
loss_fn = DiceLoss()
optimizer = torch.optim.Adam(model.parameters())

In [251]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(train_loader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [252]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 2

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)


    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(valid_loader):
            vinputs, vlabels = vdata
            voutputs = model(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:


RuntimeError: Input type (CUDABoolType) and weight type (torch.cuda.FloatTensor) should be the same

In [ ]:
class UNet(nn.Module):
    def __init__(self, n_class):
        super().__init__()
        
        # Encoder
        # In the encoder, convolutional layers with the Conv2d function are used to extract features from the input image. 
        # Each block in the encoder consists of two convolutional layers followed by a max-pooling layer, with the exception of the last block which does not include a max-pooling layer.
        # -------
        # input: 572x572x3
        self.e11 = nn.Conv2d(3, 64, kernel_size=3, padding=1) # output: 570x570x64
        self.e12 = nn.Conv2d(64, 64, kernel_size=3, padding=1) # output: 568x568x64
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # output: 284x284x64

        # input: 284x284x64
        self.e21 = nn.Conv2d(64, 128, kernel_size=3, padding=1) # output: 282x282x128
        self.e22 = nn.Conv2d(128, 128, kernel_size=3, padding=1) # output: 280x280x128
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # output: 140x140x128

        # input: 140x140x128
        self.e31 = nn.Conv2d(128, 256, kernel_size=3, padding=1) # output: 138x138x256
        self.e32 = nn.Conv2d(256, 256, kernel_size=3, padding=1) # output: 136x136x256
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2) # output: 68x68x256

        # input: 68x68x256
        self.e41 = nn.Conv2d(256, 512, kernel_size=3, padding=1) # output: 66x66x512
        self.e42 = nn.Conv2d(512, 512, kernel_size=3, padding=1) # output: 64x64x512
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2) # output: 32x32x512

        # input: 32x32x512
        self.e51 = nn.Conv2d(512, 1024, kernel_size=3, padding=1) # output: 30x30x1024
        self.e52 = nn.Conv2d(1024, 1024, kernel_size=3, padding=1) # output: 28x28x1024


        # Decoder
        self.upconv1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.d11 = nn.Conv2d(1024, 512, kernel_size=3, padding=1)
        self.d12 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        self.upconv2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.d21 = nn.Conv2d(512, 256, kernel_size=3, padding=1)
        self.d22 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        self.upconv3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.d31 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.d32 = nn.Conv2d(128, 128, kernel_size=3, padding=1)

        self.upconv4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.d41 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.d42 = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        # Output layer
        self.outconv = nn.Conv2d(64, n_class, kernel_size=1)

        def forward(self, x):
            # Encoder
            xe11 = relu(self.e11(x))
            xe12 = relu(self.e12(xe11))
            xp1 = self.pool1(xe12)

            xe21 = relu(self.e21(xp1))
            xe22 = relu(self.e22(xe21))
            xp2 = self.pool2(xe22)

            xe31 = relu(self.e31(xp2))
            xe32 = relu(self.e32(xe31))
            xp3 = self.pool3(xe32)

            xe41 = relu(self.e41(xp3))
            xe42 = relu(self.e42(xe41))
            xp4 = self.pool4(xe42)

            xe51 = relu(self.e51(xp4))
            xe52 = relu(self.e52(xe51))

            # Decoder
            xu1 = self.upconv1(xe52)
            xu11 = torch.cat([xu1, xe42], dim=1)
            xd11 = relu(self.d11(xu11))
            xd12 = relu(self.d12(xd11))

            xu2 = self.upconv2(xd12)
            xu22 = torch.cat([xu2, xe32], dim=1)
            xd21 = relu(self.d21(xu22))
            xd22 = relu(self.d22(xd21))

            xu3 = self.upconv3(xd22)
            xu33 = torch.cat([xu3, xe22], dim=1)
            xd31 = relu(self.d31(xu33))
            xd32 = relu(self.d32(xd31))

            xu4 = self.upconv4(xd32)
            xu44 = torch.cat([xu4, xe12], dim=1)
            xd41 = relu(self.d41(xu44))
            xd42 = relu(self.d42(xd41))

            # Output layer
            out = self.outconv(xd42)

            return out

In [ ]:



# выбор функции активацииЖ
activation_funk = 'relu'
# activation_funk = 'tanh'
# activation_funk = 'sigmoid'
# activation_funk = leaky_relu_layer

conv_dim= [16, 32, 64]
conv_size  = [40, 40, 20]
maxpool_size = [5, 5, 5]

dpout = 0.2



metrix = dice1_metric
metrix_loss = dice1_loss

# metrix = dice_modified
# metrix_loss = dice_modified_loss

# metrix = dice_symetr1
# metrix_loss = dice_symetr1_loss

# metrix_loss = dice_cuted_loss
# metrix = dice_cuted

class Unet(nn.Module):
  def __init__(self, input_length, input_channels):
      super().__init__()
    

      self.model = Sequential()
      self.input_length = input_length
      self.input_channels = input_channels
      self.tensorboard_callback = None

  def build_model(self):
      # Encoder
              # Define input tensor
      inputs = Input(shape=(self.input_length, self.input_channels))

      # Encoder
      encoder1 = Conv1D(conv_dim[0], conv_size[0], activation=activation_funk, strides=1, padding='same')(inputs)
      # encoder1 = Conv1D(conv_dim[0], conv_size[0], activation=activation_funk, strides=1, padding='same')(encoder1)
      # encoder1 = Conv1D(conv_dim[0], conv_size[0], activation=activation_funk, strides=1, padding='same')(encoder1)
      encoder1_pool = MaxPooling1D(maxpool_size[0])(encoder1)
      encoder1_dropout = Dropout(dpout)(encoder1_pool)

      encoder2 = Conv1D(conv_dim[1], conv_size[1], activation=activation_funk, padding='same')(encoder1_dropout)
      # encoder2 = Conv1D(conv_dim[1], conv_size[1], activation=activation_funk, padding='same')(encoder2)
      # encoder2 = Conv1D(conv_dim[1], conv_size[1], activation=activation_funk, padding='same')(encoder2)
      encoder2_pool = MaxPooling1D(maxpool_size[1])(encoder2)

      encoder3 = Conv1D(conv_dim[2], conv_size[2], activation=activation_funk, padding='same')(encoder2_pool)
      # encoder3 = Conv1D(conv_dim[2], conv_size[2], activation=activation_funk, padding='same')(encoder3)
      # encoder3 = Conv1D(conv_dim[2], conv_size[2], activation=activation_funk, padding='same')(encoder3)
      encoder3_pool = MaxPooling1D(maxpool_size[2])(encoder3)

      # Decoder
      decoder1 = Conv1DTranspose(conv_dim[2], conv_size[2], activation=activation_funk, padding='same')(encoder3_pool)
      # decoder1 = Conv1DTranspose(conv_dim[2], conv_size[2], activation=activation_funk, padding='same')(decoder1)
      # decoder1 = Conv1DTranspose(conv_dim[2], conv_size[2], activation=activation_funk, padding='same')(decoder1)
      decoder1_upsample = UpSampling1D(maxpool_size[2])(decoder1)
      decoder1_concat = concatenate([decoder1_upsample, encoder3])

      decoder2 = Conv1DTranspose(conv_dim[1], conv_size[1], activation=activation_funk, padding='same')(decoder1_concat)
      # decoder2 = Conv1DTranspose(conv_dim[1], conv_size[1], activation=activation_funk, padding='same')(decoder2)
      # decoder2 = Conv1DTranspose(conv_dim[1], conv_size[1], activation=activation_funk, padding='same')(decoder2)
      decoder2_upsample = UpSampling1D(maxpool_size[1])(decoder2)

      decoder2_dropout = Dropout(dpout)(decoder2_upsample)
      decoder2_concat = concatenate([decoder2_dropout, encoder2])

      decoder3 = Conv1DTranspose(conv_dim[0], conv_size[0], activation=activation_funk, padding='same')(decoder2_concat)
      # decoder3 = Conv1DTranspose(conv_dim[0], conv_size[0], activation=activation_funk, padding='same')(decoder3)
      # decoder3 = Conv1DTranspose(conv_dim[0], conv_size[0], activation=activation_funk, padding='same')(decoder3)
      decoder3_upsample = UpSampling1D(maxpool_size[0])(decoder3)

      decoder3_concat = concatenate([decoder3_upsample, encoder1])

      output = Conv1D(1, 1, activation='sigmoid', padding='same')(decoder3_concat)


      # Define model with input and output tensors
      self.model = tf.keras.Model(inputs=inputs, outputs=output)


      # Compile the model with dice loss and metric
      self.model.compile(optimizer='adam', loss=metrix_loss, metrics=[metrix])

      # # Create a TensorBoard callback
      self.tensorboard_callback = TensorBoard(log_dir='./logs')



  def summary(self):
      self.model.summary()

  def train(self, x_train, y_train, x_val, y_val, epochs, batch_size):

      log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
      tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
      self.model.fit (
          x_train,
          y_train,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(x_val, y_val),
          callbacks=[tensorboard_callback]

          )


# Create an instance of the Autoencoder class
autoencoder = Autoencoder(input_length, input_channels)
autoencoder.build_model()
autoencoder.summary()


def forward_pass(model, x):
    # Преобразование входных данных в тензор TensorFlow
    x_tensor = tf.convert_to_tensor(x, dtype=tf.float32)

    # Выполнение прямого прохода
    y_pred = model(x_tensor)

    # Преобразование предсказания в массив NumPy
    y_pred_np = y_pred.numpy()

    return y_pred_np

In [ ]:
from torch.utils.data import DataLoader

batch_size = 128

train_loader = DataLoader(x_train, batch_size=batch_size, drop_last=True)
test_loader = DataLoader(y_train, batch_size=batch_size, drop_last=True)

In [43]:
from tensorflow.keras.callbacks import TensorBoard

from tensorflow.summary import create_file_writer
# Load the TensorBoard notebook extension
%load_ext tensorboard

import tensorflow as tf
import datetime

from torch.utils.tensorboard import SummaryWriter # Запись логов в тензорборд
writer = SummaryWriter() # Создание объекта для записи

In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Conv1DTranspose, UpSampling1D, Dropout
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Conv1DTranspose, UpSampling1D
from tensorflow.keras.layers import concatenate, Input

In [ ]:



# выбор функции активацииЖ
activation_funk = 'relu'
# activation_funk = 'tanh'
# activation_funk = 'sigmoid'
# activation_funk = leaky_relu_layer

conv_dim= [16, 32, 64]
conv_size  = [40, 40, 20]
maxpool_size = [5, 5, 5]

dpout = 0.2



metrix = dice1_metric
metrix_loss = dice1_loss

# metrix = dice_modified
# metrix_loss = dice_modified_loss

# metrix = dice_symetr1
# metrix_loss = dice_symetr1_loss

# metrix_loss = dice_cuted_loss
# metrix = dice_cuted

class Autoencoder:
  def __init__(self, input_length, input_channels):
      self.model = Sequential()
      self.input_length = input_length
      self.input_channels = input_channels
      self.tensorboard_callback = None

  def build_model(self):
      # Encoder
              # Define input tensor
      inputs = Input(shape=(self.input_length, self.input_channels))

      # Encoder
      encoder1 = Conv1D(conv_dim[0], conv_size[0], activation=activation_funk, strides=1, padding='same')(inputs)
      # encoder1 = Conv1D(conv_dim[0], conv_size[0], activation=activation_funk, strides=1, padding='same')(encoder1)
      # encoder1 = Conv1D(conv_dim[0], conv_size[0], activation=activation_funk, strides=1, padding='same')(encoder1)
      encoder1_pool = MaxPooling1D(maxpool_size[0])(encoder1)
      encoder1_dropout = Dropout(dpout)(encoder1_pool)

      encoder2 = Conv1D(conv_dim[1], conv_size[1], activation=activation_funk, padding='same')(encoder1_dropout)
      # encoder2 = Conv1D(conv_dim[1], conv_size[1], activation=activation_funk, padding='same')(encoder2)
      # encoder2 = Conv1D(conv_dim[1], conv_size[1], activation=activation_funk, padding='same')(encoder2)
      encoder2_pool = MaxPooling1D(maxpool_size[1])(encoder2)

      encoder3 = Conv1D(conv_dim[2], conv_size[2], activation=activation_funk, padding='same')(encoder2_pool)
      # encoder3 = Conv1D(conv_dim[2], conv_size[2], activation=activation_funk, padding='same')(encoder3)
      # encoder3 = Conv1D(conv_dim[2], conv_size[2], activation=activation_funk, padding='same')(encoder3)
      encoder3_pool = MaxPooling1D(maxpool_size[2])(encoder3)

      # Decoder
      decoder1 = Conv1DTranspose(conv_dim[2], conv_size[2], activation=activation_funk, padding='same')(encoder3_pool)
      # decoder1 = Conv1DTranspose(conv_dim[2], conv_size[2], activation=activation_funk, padding='same')(decoder1)
      # decoder1 = Conv1DTranspose(conv_dim[2], conv_size[2], activation=activation_funk, padding='same')(decoder1)
      decoder1_upsample = UpSampling1D(maxpool_size[2])(decoder1)
      decoder1_concat = concatenate([decoder1_upsample, encoder3])

      decoder2 = Conv1DTranspose(conv_dim[1], conv_size[1], activation=activation_funk, padding='same')(decoder1_concat)
      # decoder2 = Conv1DTranspose(conv_dim[1], conv_size[1], activation=activation_funk, padding='same')(decoder2)
      # decoder2 = Conv1DTranspose(conv_dim[1], conv_size[1], activation=activation_funk, padding='same')(decoder2)
      decoder2_upsample = UpSampling1D(maxpool_size[1])(decoder2)

      decoder2_dropout = Dropout(dpout)(decoder2_upsample)
      decoder2_concat = concatenate([decoder2_dropout, encoder2])

      decoder3 = Conv1DTranspose(conv_dim[0], conv_size[0], activation=activation_funk, padding='same')(decoder2_concat)
      # decoder3 = Conv1DTranspose(conv_dim[0], conv_size[0], activation=activation_funk, padding='same')(decoder3)
      # decoder3 = Conv1DTranspose(conv_dim[0], conv_size[0], activation=activation_funk, padding='same')(decoder3)
      decoder3_upsample = UpSampling1D(maxpool_size[0])(decoder3)

      decoder3_concat = concatenate([decoder3_upsample, encoder1])

      output = Conv1D(1, 1, activation='sigmoid', padding='same')(decoder3_concat)


      # Define model with input and output tensors
      self.model = tf.keras.Model(inputs=inputs, outputs=output)


      # Compile the model with dice loss and metric
      self.model.compile(optimizer='adam', loss=metrix_loss, metrics=[metrix])

      # # Create a TensorBoard callback
      self.tensorboard_callback = TensorBoard(log_dir='./logs')



  def summary(self):
      self.model.summary()

  def train(self, x_train, y_train, x_val, y_val, epochs, batch_size):

      log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
      tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
      self.model.fit (
          x_train,
          y_train,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(x_val, y_val),
          callbacks=[tensorboard_callback]

          )


# Create an instance of the Autoencoder class
autoencoder = Autoencoder(input_length, input_channels)
autoencoder.build_model()
autoencoder.summary()


def forward_pass(model, x):
    # Преобразование входных данных в тензор TensorFlow
    x_tensor = tf.convert_to_tensor(x, dtype=tf.float32)

    # Выполнение прямого прохода
    y_pred = model(x_tensor)

    # Преобразование предсказания в массив NumPy
    y_pred_np = y_pred.numpy()

    return y_pred_np